In [1]:
import tensorflow as tf
import random
import math
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
#np.set_printoptions(threshold=np.inf)
import time
import csv

# File Name setting

In [2]:
##tr 0 to 3 is location value
filename = [ "wifi_1462533845","wifi_1462534198","wifi_1462581614","wifi_1462581861","wifi_1462582182"
            ,"wifi_1462582432","wifi_1462582667","wifi_1462583580","wifi_1462586623"
            ,"wifi_1462587384","wifi_1462589535","wifi_1462589884","wifi_1462590276","wifi_1462594202"
            ,"wifi_1462595833","wifi_1462598045","wifi_1462599051","wifi_1462599292"
            ,"wifi_1462599796","wifi_1462600700","wifi_1462603000","wifi_1462603364","wifi_1462671495"
            ,"wifi_ET_3_1472725261","wifi_ET_3_1472725954","wifi_SH_2_1472731249","wifi_SH_2_1472731992"
            ,"wifi_SL_3_1472726646","wifi_SL_3_1472727115","wifi_SL_3_1472727610","wifi_TV_3_1472728122"
            ,"wifi_TV_3_1472728582","wifi_TV_3_1472729076","wifi_VC_3_1472729944","wifi_VC_3_1472730571"
            ,"wifi_ET_3_1472724912","wifi_ET_3_1472725609","wifi_SH_2_1472730943","wifi_SH_2_1472731635"
            ,"wifi_SL_3_1472726399","wifi_SL_3_1472726878","wifi_SL_3_1472727374","wifi_TV_3_1472727893"
            ,"wifi_TV_3_1472728356","wifi_TV_3_1472728861","wifi_VC_3_1472729626","wifi_VC_3_1472730262"
            ,"wifi_1467699078","wifi_1467699827","wifi_1467700220","wifi_1467700540","wifi_1467701698"
            ,"wifi_1467701925","wifi_1467702155","wifi_1467698011","wifi_1467698236","wifi_1467698473"
            ,"wifi_1467696862","wifi_1467697617","wifi_1467698757","wifi_1467699496","wifi_1467694981"
            ,"wifi_1467695357","wifi_1467700994","wifi_1467701219","wifi_1467695809","wifi_1467696035"
            ,"wifi_1467696259","wifi_1467696542","wifi_1467697281","wifi_ET_0_1472546966","wifi_ET_1_1472554884"
            ,"wifi_ET_2_1472551231","wifi_ET_3_1472564959", "wifi_ET_3_1420108956", "wifi_ET_2_1420095231"
            ,"wifi_ET_1_1420098884","wifi_ET_0_1420090965","wifi_ET_3_1420134169","wifi_ET_2_1420142476"
            ,"wifi_ET_1_1420137839","wifi_ET_0_1420114448","wifi_ET_3_1472590170","wifi_ET_2_1472598476"
            ,"wifi_ET_1_1472593839","wifi_ET_0_1472570446"]

filename_te = "5Act_Var_Tr2_HC_Wifi_raw_complete"

# Variable setting

In [3]:
training_step = 3000
learning_rate = 0.001

# Constant variable

In [4]:
location = []

ap_address = []
ap_signal = []

bi_location = []
ap_lev = []
ap_lev_times = []
selected_ap = []

# Function ==================================

file은 한번 만 읽어서 각 array에 저장한 후에 처리하도록 함(같은 파일을 다시 읽지 않음)

In [5]:
def read_file(filename,ap_address, ap_signal, location):
    filename = "data/wifi_et/"+filename+".csv"
    with open(filename, 'rb') as csvfile:
        reader = csv.reader (csvfile, delimiter = ',')
        for row in reader:
            ap_address.append(row[2])
            ap_signal.append(row[3])
            location.append(row[4])

총 ap_address 들을 구해줌

In [6]:
def get_ap_lev(ap_address,ap_lev):
    for i in range(0,len(ap_address)):
        if ap_address[i] not in ap_lev:
            ap_lev.append(ap_address[i])
    return ap_lev

해당 ap가 총 몇 회 불렸는지 확인하는 함수 해당 함수는 get_ap_lev 함수 실행 이후에 돌려야함
count함수와 같음

In [7]:
def get_ap_levtimes(ap_address,ap_lev,ap_lev_times):
    ap_lev_times = np.zeros(len(ap_lev),dtype=np.int)
    for i in range(0, len(ap_address)):
        index = ap_lev.index(ap_address[i])
        ap_lev_times[index] += 1
    return ap_lev_times

ap_signal의 등장횟수가 많은 것을 기준으로 NN에 이용할 AP를 선택하는 함수

In [8]:
def select_ap_address(data_max,ap_lev, ap_lev_times, selected_ap):
    selected_ap_index = []
    threshold = 7330
    for i in range(0,len(ap_lev_times)):
        if ap_lev_times[i] >= threshold:
            selected_ap_index.append(i)
    for i in range(0, len(selected_ap_index)):
        selected_ap.append(ap_lev[selected_ap_index[i]])
    return threshold

data의 수가 가장 많은 것을 기준으로, 해당 data의 등장후 다음 등장 이전까지를 하나의 epoch으로 지정

위의 흐름대로 진행할 때 데이터 중에서 다른 location 데이터와 함께 들어와 있는 경우, 
무조건 행에 따라서 data를 나열하면 input(ap_signal)과 output(location)에 혼선이 있을 우려가 있음

[0 1 2 0 1 2 0 1] 로 등장하였다고 가정하고, 해당 하는 location 이<br>[5 5 5 5 5 5 5 4] 로 등장하였을 경우, 

마지막 두 개의 ap_address_index에서 0 1 이 다른 location인 경우가 발생되는 것을 방지하고자 함

우선 (location 별로 partition을 생성한 다음)==>raw data는 원래 떨어져 있으므로, 이것을 기준으로 다시 짜도록함.

각 파티션 별로 ap selected array를 구현한 다음 다시 합쳐지는 방식을 이용함


1. (선택된 ap index) x (data_max) 행렬
2. (location) x (data_max) 행렬

해당 함수는 selected_ap_signal의 갯수가 4개임을 알고서 하는 것.

수정이 요구되나,<br>
4개만 있어도 결과값이 잘 나온다면 애초에 셀렉트할때 4개만 해도 무방할 것이라고 판단됨.

In [9]:
def convert_to_selected_ap_array(ap_address,ap_signal,selected_ap,location, data_max):
    j = -1
    converted_ap_signal = np.zeros((data_max, len(selected_ap)))
    converted_location = []
    for i in range(0,len(ap_signal)):
        if ap_address[i] == selected_ap[0]:
            j = j+1
            converted_ap_signal[j][0] = ap_signal[i]
            converted_location.append(location[i])
        elif ap_address[i] == selected_ap[1]:
            converted_ap_signal[j][1] = ap_signal[i]
        else :
            continue
    return converted_ap_signal, converted_location

In [10]:
def use_before_data_if_val_0(ap_signal,data_max, selected_ap):
    selected_ap_num = len(selected_ap)
    for i in range(1,data_max):
        for j in range(0,selected_ap_num):
            if ap_signal[i][j] == 0 :
                ap_signal[i][j] = ap_signal[i-1][j]

location을 softmax함수에 이용할 수 있도록 변환 <br>
1 => [0,1,0,0]<br>
3 => [0,0,0,1] 식으로 변환하는 함수

In [11]:
def make_location_to_binary_array(data_max,location):
    if data_max != len(location):
        print "Miss matched!"
        return 
    
    location_num = int(location[np.argmax(location)])
    bi_location = np.zeros((data_max,location_num+1), dtype = int)
    for i in range(0,data_max):
        bi_location[i,location[i]] = 1
    return bi_location

weight, bias 생성 함수 y = wx + b

In [12]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# ================================Function End

# Training data pre_processing

ap의 level과 총 level갯수

In [13]:
for i in range(0,len(filename)):
    read_file(filename[i],ap_address, ap_signal, location)
    
get_ap_lev(ap_address,ap_lev)
ap_lev_times = get_ap_levtimes(ap_address,ap_lev,ap_lev_times)
data_max = ap_lev_times[np.argmax(ap_lev_times)]
threshold = select_ap_address(data_max,ap_lev, ap_lev_times, selected_ap)
converted_ap_signal, converted_location = convert_to_selected_ap_array(ap_address,ap_signal,selected_ap,location, data_max)
bi_location = make_location_to_binary_array(data_max, converted_location)

/home/dclabyeop/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


use_before_data_if_val_0(converted_ap_signal, data_max, selected_ap)

# Print training values

In [14]:
print get_ap_lev(ap_address, ap_lev)
print len(ap_lev)

['00:26:66:d0:c4:dc', '00:27:1c:ea:40:af', '00:26:66:76:37:9c', '00:27:1c:08:41:16', '00:01:36:2d:ea:4c', '90:9f:33:6d:8e:b6', '00:23:aa:3d:d9:41', '00:30:0d:62:4c:24', '0a:0e:dc:5a:b6:c8', '64:e5:99:a3:4d:5c', '00:40:5a:db:5e:c9', '90:9f:33:40:a0:9e', '64:e5:99:82:5a:20', '64:e5:99:82:5a:21', '08:5d:dd:2d:a4:a8', '90:9f:33:bb:16:68', '10:f9:6f:f4:11:1d', '00:08:5b:80:8e:f1', '64:e5:99:4d:7b:68', '64:e5:99:e6:18:04', '00:23:aa:3e:a3:a9', '00:08:5b:5a:20:59', '00:07:89:2b:53:35', '90:9f:33:6c:a4:92', '00:23:aa:3c:7a:c1', '00:08:52:40:4f:df', '08:5d:dd:71:40:b4', '00:26:66:f7:fc:34', '00:26:66:5c:f7:f0', '04:8d:39:36:e5:9d', '04:8d:38:95:b3:f5', '00:4e:00:0e:c0:a8', '00:07:89:55:b0:86', '00:27:1c:ETVa:40:af', '00:01:36:2d:ETVa:4c', '0a:0ETV:dc:5a:b6:c8', '00:08:9f:16:28:42', '00:40:5a:db:5ETV:c9', '64:66:b3:3a:3a:73', '90:9f:33:b5:24:b2', '00:26:66:78:99:a8', '08:5d:dd:0b:0a:3a', '64:ETV5:99:82:5a:20', '90:9f:33:40:a0:9ETV', '90:9f:33:8f:a2:6a', '00:ETV0:4c:81:86:86', '00:40:5a:e2:02:f1'

In [15]:
print ap_lev_times
print len(ap_lev_times)

[7586 7127 2805 7331 7104  193 3154 1145   72   19  406  412   50  529   29
   15   16   17   15    2    4    4   25    4   38    2 1348    7    8  540
    8    6    2  123  132   18  128    9    2    4  367   12    4    2  318
    2    2  296  261  110   52    3    4    2    1   69   46   34  113  185
    4]
61


In [16]:
print threshold
print selected_ap

7330
['00:26:66:d0:c4:dc', '00:27:1c:08:41:16']


In [17]:
print len(converted_location)

7586


# Training data set up

In [18]:
print converted_ap_signal

[[-63. -85.]
 [-66. -85.]
 [-63. -84.]
 ..., 
 [-49. -71.]
 [-58. -74.]
 [-49. -76.]]


In [19]:
print bi_location

[[0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 ..., 
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]


In [20]:
location_num = int(location[np.argmax(location)]) + 1 # 0, 1, 2, 3
selected_num = len(selected_ap)
hidden_dim = selected_num

In [21]:
print location_num
print selected_num

4
2


In [22]:
tr_x = converted_ap_signal

In [23]:
tr_y = bi_location

# Variable reset

In [24]:
ap_signal = []
ap_address = []
location = []
ap_lev = []
ap_lev_times = []
input_num = data_max
data_max = 0

# Testing data set up

In [25]:
read_file(filename_te,ap_address, ap_signal, location)

get_ap_lev(ap_address,ap_lev)
ap_lev_times = get_ap_levtimes(ap_address,ap_lev,ap_lev_times)
data_max = ap_lev_times[np.argmax(ap_lev_times)]
converted_ap_signal, converted_location = convert_to_selected_ap_array(ap_address,ap_signal,selected_ap,location, data_max)
bi_location = make_location_to_binary_array(data_max, converted_location)

/home/dclabyeop/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


use_before_data_if_val_0(converted_ap_signal, data_max, selected_ap)

In [26]:
print converted_ap_signal

[[-77. -97.]
 [-73. -95.]
 [-72. -95.]
 [-73. -93.]
 [-73. -91.]
 [-70. -94.]
 [-71. -92.]
 [-69. -94.]
 [-73. -89.]
 [-73. -91.]
 [-72. -95.]
 [-71. -93.]
 [-72. -91.]
 [-67. -91.]
 [-72. -91.]
 [-74. -91.]
 [-65. -90.]
 [-68. -93.]
 [-69. -93.]
 [-68. -91.]
 [-71. -91.]
 [-69. -89.]
 [-66. -90.]
 [-67. -90.]
 [-71. -86.]
 [-69. -86.]
 [-68. -92.]
 [-70. -91.]
 [-71. -87.]
 [-75. -89.]
 [-68. -86.]
 [-67. -91.]
 [-66. -88.]
 [-66. -85.]
 [-69. -86.]
 [-71. -91.]
 [-75. -89.]
 [-78. -89.]
 [-75. -89.]
 [-77. -90.]
 [-73. -87.]
 [-74. -91.]
 [-75. -91.]
 [-77. -89.]
 [-73. -89.]
 [-76. -88.]
 [-75. -88.]
 [-74. -93.]
 [-78. -91.]
 [-69. -91.]
 [-70. -91.]
 [-72. -91.]
 [-66. -88.]
 [-68. -87.]
 [-68. -88.]
 [-66. -89.]
 [-67. -89.]
 [-70. -90.]
 [-70. -89.]
 [-76. -91.]
 [-70. -90.]
 [-69. -89.]
 [-68. -90.]
 [-69. -90.]
 [-72. -90.]
 [-69. -93.]
 [-74. -88.]
 [-71. -91.]
 [-75. -92.]
 [-74. -92.]
 [-55. -75.]
 [-69. -76.]
 [-58. -84.]
 [-55. -88.]
 [-59. -89.]
 [-57. -89.]
 [-65. -91.]

In [27]:
print bi_location

[[0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 ..., 
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]


In [28]:
te_x = converted_ap_signal

In [29]:
te_y = bi_location

In [30]:
print selected_ap
print data_max

['00:26:66:d0:c4:dc', '00:27:1c:08:41:16']
296


# Neural network algorithm : input ap signal layer +2 hidden layer+softmax

In [31]:
hidden_dim =  int(math.sqrt((location_num+2)*input_num)+2*math.sqrt(input_num/location_num+2))
hidden_dim2 =  int(location_num*math.sqrt(input_num/location_num+2))

W = weight_variable([selected_num, hidden_dim])
b = bias_variable([hidden_dim])

W2 = weight_variable([hidden_dim,hidden_dim2])
b2 = bias_variable([hidden_dim2])

W3 = weight_variable([hidden_dim2,location_num])
b3 = bias_variable([location_num])

X = tf.placeholder("float",[None, selected_num])
Y = tf.placeholder("float",[None,location_num])

hidden = tf.nn.relu(tf.matmul(X, W)+b)
hidden2 = tf.nn.relu(tf.matmul(hidden,W2)+b2)
sfmax = tf.nn.softmax(tf.matmul(hidden2,W3)+b3)

cross_entropy =tf.reduce_mean(-tf.reduce_sum(Y*tf.log(sfmax + 1e-10)))
train = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(sfmax,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#AdagradOptimizer , AdamOptimizer, GradientDescentOptimizer

In [32]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

start_time = time.time()
for step in range(training_step+1):
    result = sess.run(train, feed_dict = {X: tr_x, Y: tr_y})
    if (step % 200) == 0:
        print sess.run(accuracy,feed_dict = {X: te_x, Y: te_y})*100

print sess.run(accuracy,feed_dict = {X: te_x, Y: te_y})*100
excute_time = time.time() - start_time
sess.close()
print excute_time

23.6486494541
52.0270287991
53.3783793449
56.0810804367
54.3918967247
57.7702701092
57.7702701092
55.7432472706
57.0945978165
58.1081092358
52.7027070522
57.4324369431
57.0945978165
57.0945978165
53.7162125111
53.7162125111
53.716224432
58.5904991627
